# Assignment 2

**Objective:**  
Create and visualize three different types of graphs using the datasets provided:  
- **bar_assignment.csv**  
- **sankey_assignment.csv**  
- **networks_assignment.csv**

**General Instructions:**  
Use the same font and font size for all graphs. Ensure that you adhere to the plot specifications and color schemes provided for each graph.

---

## Tasks

### A. Bar Graph:
Create a **horizontal stacked bar chart**.
- **Transform data:**
  - Convert `1` into “Yes” and `0` into “No”.
- **Plot Specification:**  
  - Follow the instructions to display the bar chart as a horizontal stacked bar.

### B. Sankey Diagram:
Create a **Sankey Diagram** that connects the following:
- Source Nodes: ('PS', 'OMP', 'CNP', 'NRP', 'NMCCC', 'PEC', 'NCDM', 'RGS')
- Target Nodes: ('Reg', 'Aca', 'Oth')

Follow the **Path Specifications** to properly connect the nodes in the diagram.

### C. Network Graph:
Create a **network graph** with specific node placements:
- **Pentagram Structure:**  
  Nodes `D`, `F`, `I`, `N`, `S` should form a pentagram at the center of the graph. Ensure they show connections to each other.
- **Outer Nodes:**  
  Other nodes should be positioned outside the pentagram but still show connections to other nodes.
- **Node Colors:**
  - **Blue:** `['D', 'F', 'I', 'N', 'S']`
  - **Green:** `['BIH', 'GEO', 'ISR', 'MNE', 'SRB', 'CHE', 'TUR', 'UKR', 'GBR', 'AUS', 'HKG', 'USA']`
  - **Yellow:** `['AUT', 'BEL', 'BGR', 'HRV', 'CZE', 'EST', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LUX', 'NLD', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP']`

### Final Task:
1. **Collate the Graphs:**  
   Combine all three graphs into a single graph that is optimized for printing.  
   - This final graph should be visible when copy-pasted into a long bond paper.

---

## Deliverables:

1. **Code**  
   - Provide the code for all three tasks.

2. **Individual Graphs:**
   - PNG files for each graph (Bar Graph, Sankey Diagram, Network Graph).

3. **Collated Graph:**
   - PNG file of the collated graphs.

4. **Sample Collated File:**
   - A sample of the collated file pasted into a document and saved as a `.pdf`.

5. **GitHub Submission:**
   - Push all files to your GitHub account.

